In [1]:
import numpy as np
import pandas as pd
import deepxde as dde
from deepxde.backend import tf


2023-10-02 19:29:32.539467: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-02 19:29:32.558075: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-02 19:29:32.558092: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-02 19:29:32.558103: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-02 19:29:32.562084: I tensorflow/core/platform/cpu_feature_g

Instructions for updating:
non-resource variables are not supported in the long term


2023-10-02 19:29:33.208742: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-02 19:29:33.222272: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-02 19:29:33.222387: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Enable just-in-time compilation with XLA.



In [2]:
df_Cin_conocido = pd.read_excel("datos_cromatografia.xlsx", 0)
Cobs_conocido = df_Cin_conocido['AU']
t_conocido = df_Cin_conocido['min']

# Escalado de datos
t_min, t_max = t_conocido.min(), t_conocido.max()
Cobs_min, Cobs_max = Cobs_conocido.min(), Cobs_conocido.max()
t_scaled = (t_conocido - t_min) / (t_max - t_min)
Cobs_scaled = (Cobs_conocido - Cobs_min) / (Cobs_max - Cobs_min)


In [3]:
# Parámetros y funciones
epsilon = 0.37
u = 5.55
L = 15
tp = 1.0

# Función de equilibrio
def q(C, K):
    return K * C

# Definición de la EDP
def pde(t, C):
    dC_dt = dde.grad.jacobian(C, t)
    dC_dt2 = dde.grad.hessian(C, t)
    return dC_dt + (1 - epsilon) / epsilon * q(C, K) + u / epsilon * dC_dt - DL * dC_dt2

# Condiciones iniciales y de frontera
geom = dde.geometry.Interval(t_min, t_max)
bc = dde.DirichletBC(geom, lambda x: 0, lambda _, on_boundary: on_boundary)


In [4]:
data = dde.data.TimePDE(geom, pde, bc, num_domain=1000, num_boundary=100, solution=None, num_test=None)

net = dde.maps.FNN([1] + [50] * 3 + [1], "tanh", "Glorot uniform")
model = dde.Model(data, net)
model.compile("adam", lr=0.001)
losshistory, train_state = model.train(epochs=10000)


Compiling model...
Building feed-forward neural network...
'build' took 0.017637 s



/home/vasco/dev/Python-Projects/HackSciML-g19/venv/lib/python3.10/site-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(
2023-10-02 19:29:37.827788: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-02 19:29:37.827886: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-02 19:29:37.827926: I tensorflow

NameError: name 'K' is not defined

In [5]:
# Obtener todas las variables entrenables
trainable_vars = tf.trainable_variables()

# Suponiendo que DL y K son las últimas dos variables en la lista:
DL_value = model.sess.run(trainable_vars[-2])
K_value = model.sess.run(trainable_vars[-1])

print(DL_value, K_value)

2023-10-02 19:29:44.056227: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2023-10-02 19:29:44.091270: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10490655046396620328


FailedPreconditionError: Graph execution error:

2 root error(s) found.
  (0) FAILED_PRECONDITION: Attempting to use uninitialized value dense_3/kernel
	 [[{{node dense_3/kernel/_0}}]]
	 [[dense_3/kernel/_1]]
  (1) FAILED_PRECONDITION: Attempting to use uninitialized value dense_3/kernel
	 [[{{node dense_3/kernel/_0}}]]
0 successful operations.
0 derived errors ignored.

In [6]:
for var in trainable_vars:
    print(var.name, var.shape)

dense/kernel:0 (1, 50)
dense/bias:0 (50,)
dense_1/kernel:0 (50, 50)
dense_1/bias:0 (50,)
dense_2/kernel:0 (50, 50)
dense_2/bias:0 (50,)
dense_3/kernel:0 (50, 1)
dense_3/bias:0 (1,)


In [29]:
DL_value = [v for v in trainable_vars if v.name == "DL_2:0"][0]
K_value = [v for v in trainable_vars if v.name == "K_2:0"][0]

DL_value = model.sess.run(DL_value)
K_value = model.sess.run(K_value)

print(DL_value, K_value)

1.0 1.0
